1. [20 pts] In this assignment, we will approach the problem with sentiment analysis in our pipeline to classify movie reviews.
Use the Opinion Lexicon from the source: http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html
Download the two files, positive-words.txt and negative-words.txt and use them to opinion-score each pooled review (i.e., ignore the original class labels). One approach can be counting the number of occurrences of the lexicon words and scoring appropriately.
(Hint: Quantize the positive and negative frequencies to 0 and 1,
i.e., predict = (opinion_score[:,0] > opinion_score[:,1]) + 0
)
What is the prediction accuracy when done like in the above?

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
import re
import nltk
# Import necessary libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

In [2]:
import csv
def read_file(file_path,sentiment):
    start_line = 32
    words=[]

    with open(file_path, 'r', encoding="latin-1") as file:
        reader = csv.reader(file, delimiter=',', quotechar='"')
        next(reader)
        for line_number, line in enumerate(reader, start=1):
            if line_number >= start_line:
                words.append(line[0])
    return set(words)
pos_words=read_file('./positive-words.txt',1)
neg_words=read_file('./negative-words.txt',0)
all_the_words=pos_words.union(neg_words)
all_the_words

{'insincerely',
 'reforming',
 'unhappiness',
 'well-informed',
 'ambush',
 'inaptitude',
 'dissing',
 'blur',
 'rectification',
 'revitalize',
 'sadly',
 'derogatory',
 'hate',
 'depressed',
 'gallingly',
 'smoothest',
 'paranoia',
 'ironic',
 'besmirch',
 'smile',
 'awesomeness',
 'discomfort',
 'attune',
 'inequalities',
 'stricken',
 'acclaimed',
 'pest',
 'enhances',
 'deprecate',
 'hissed',
 'lorn',
 'unaffected',
 'mangling',
 'embolden',
 'sweaty',
 'calamitously',
 'paradoxical',
 'obsessions',
 'jolt',
 'spoil',
 'threesome',
 'opinionated',
 'cheesy',
 'traction',
 'dedicated',
 'outcast',
 'haunting',
 'scornfully',
 'leakages',
 'compliant',
 'thrift',
 'purposeful',
 'beautiful',
 'liberation',
 'inattentive',
 'dejectedly',
 'ineloquent',
 'hypocritical',
 'refute',
 'ruts',
 'atrocious',
 'hypocrisy',
 'longing',
 'futile',
 'obscenely',
 'pillage',
 'noiseless',
 'pleas',
 'apathetic',
 'amply',
 'smitten',
 'complained',
 'truthfulness',
 'prefers',
 'unprepared',
 's

In [3]:
from collections import Counter
import nltk
from nltk.corpus import stopwords
import pandas as pd
import re

# Assuming you have a set of positive and negative words defined as pos_words and all_the_words

def ie_preprocess(document):
    document = re.sub('<br />', '', document)
    document = re.sub(r'[^\w\s]', '', document)
    sentences = [nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(document)]
    stop_words = set(stopwords.words('english'))
    return [[word for word in sent if word.lower() in all_the_words] for sent in sentences][0]

def count_sentiment(words):
    return Counter(1 if word.lower() in pos_words else 0 for word in words)

def predict_sentiment(counts):
    return 0 if counts[0] > counts[1] else 1 if counts[1] > counts[0] else -1

def return_pos(counts):
    return counts[1]

def return_neg(counts):
    return counts[0]

path = './movie_data.csv'
df = pd.read_csv(path, encoding="utf-8")
df['review'] = df['review'].apply(ie_preprocess)
df['counts_of_pos_neg'] = df['review'].apply(count_sentiment)
df['predicted_sentiment'] = df['counts_of_pos_neg'].apply(predict_sentiment)
df['pos'] = df['counts_of_pos_neg'].apply(return_pos)
df['neg'] = df['counts_of_pos_neg'].apply(return_neg)
df = df.drop('counts_of_pos_neg', axis=1)


In [4]:
df.head()

,review,sentiment,predicted_sentiment,pos,neg
0,"[Grace, Mischief, murder, fallen, disgrace, sq...",1,0,8,12
1,"[like, easy, helped, soft, steal, effortlessly...",0,1,13,10
2,"[waste, plot, Ugly, worth, plot, shallow, ridi...",0,0,12,20
3,"[wonderful, liked, love, fave]",1,1,4,0
4,"[hated, prominent, great, enjoyed, joy]",0,1,4,1


In [5]:
matching_count = ((df['predicted_sentiment'] == df['sentiment']).sum()/len(df))*100
print(f'Percent accuracy: {matching_count:.2f}%')

Percent accuracy: 69.39%


The Prediction Accuracy as given above is 69.39%, which is fairly good for the actual percent, considering these are using fairly rudimenary methods to determine sentiment.

2. [20 pts] Use only the first 1000 words in Tf-Idf features and compute 10-fold CV classification evaluation performance. We expect a lower score compared to when using the full set of features. Recall that when Tf-Idf features are ranked, the top features exhibit a high information content, since Idf normalizes the document frequencies properly.

In [6]:
def to_string(s):
    return str(s)

In [7]:
# Create a CountVectorizer for text data
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
# Prepare your features and labels
X_as_str=df['review'].apply(to_string)
X = tfidf_vectorizer.fit_transform(X_as_str)
y = df['sentiment']
# Transform the training data using the CountVectorizer

# Initialize and train the Logistic Regression classifier

#SVM was too slow my computer took too long
log_classifier = LinearSVC()
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(log_classifier, X, y, cv=cv, scoring='accuracy')

# Report the 10-fold cross-validation performance
mean_accuracy = np.mean(scores)
print("Mean Accuracy:", mean_accuracy)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/codespace/.local/l

Mean Accuracy: 0.84956


/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


3. [20 pts] Add the two columns of positive and negative opinions (e.g., counts) to the 1000 Tf-Idf features in (2.) and re-compute the 10-fold CV classification evaluation. Did opinions from (1.) help?

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Create a CountVectorizer for text data
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
# Prepare your features and labels

reviews = tfidf_vectorizer.fit_transform(df['review'].apply(to_string))
df_tfidf = pd.DataFrame(reviews.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
# Step 3: Add positive and negative opinions to the 1000 Tf-Idf features
df_tfidf['pos'] = df['pos']
df_tfidf['neg'] = df['neg']


X = df_tfidf.values
y = df['sentiment']

#SVM was too slow my computer took too long
log_classifier = LinearSVC()
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(log_classifier, X, y, cv=cv, scoring='accuracy')

# Report the 10-fold cross-validation performance
mean_accuracy = np.mean(scores)
print("Mean Accuracy:", mean_accuracy)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to 

Mean Accuracy: 0.8450600000000001


/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


4. [20 pts] Compare the ranked features as in the previous homework, indicate where the two added opinion features located with respect to the high-ranked Tf-Idf features (as we conducted in the previous assignments)?

In [9]:
# Extract the coefficients and feature names
log_classifier.fit(X,y)

coefficients = log_classifier.coef_[0]
feature_names = df_tfidf.columns
feature_coefficients = list(zip(feature_names, coefficients))

# Sort the feature coefficients
sorted_feature_coefficients = sorted(feature_coefficients, key=lambda x: x[1])

# Print the top and bottom coefficients
def print_top_and_bottom_coefs(sorted_feature_coefficients, num_top=10):
    print("Keywords for Best:")
    for i, (feature, coefficient) in enumerate(sorted_feature_coefficients[-num_top:][::-1]):
        print(f"#{i+1}, {feature}: {coefficient}")

    print("\nKeywords for Worst:")
    for i, (feature, coefficient) in enumerate(sorted_feature_coefficients[:num_top]):
        print(f"#{i+1}, {feature}: {coefficient}")
    print("\nSentiment Weights:")
    for i, (feature, coefficient) in enumerate(sorted_feature_coefficients):
        if(feature == 'pos'):
            print(f"#{i+1}, {feature}: {coefficient}")
        if(feature == 'neg'):
            print(f"#{i+1}, {feature}: {coefficient}")


print_top_and_bottom_coefs(sorted_feature_coefficients)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Keywords for Best:
#1, refreshing: 1.6139271836536397
#2, excellent: 1.4387734878136271
#3, heartbreaking: 1.232185132836942
#4, gem: 1.1913839484041489
#5, wonderfully: 1.1333121411406435
#6, flawless: 1.1191007105615032
#7, superb: 1.088411019969896
#8, haunting: 1.0674231440954254
#9, astonishing: 1.0347298964127385
#10, great: 1.0280151004026914

Keywords for Worst:
#1, waste: -2.4399274084038427
#2, worst: -2.3538924533938923
#3, awful: -1.965398490747878
#4, unwatchable: -1.8439213471398375
#5, wasting: -1.7915343051889248
#6, disappointment: -1.757024210843416
#7, redeeming: -1.7170643821450067
#8, fails: -1.6317664850738252
#9, poorly: -1.5870131878669311
#10, laughable: -1.5844114693776359

Sentiment Weights:
#523, neg: -0.0027406251686889864
#568, pos: 0.047287708930386446


/home/codespace/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


5. [20 pts] We were unable to improve the labeling and/or classification as it looks like the IMDB review dataset ground truth is generated from something else.
List some possibilities about the source of the ground truth of original dataset.

One possible source of the ground truth could be from the people who created the reviews themselves. These reviews could possibly be from a binary voting system along with the reviews given. One reason I personally think that the sentiment ground truth was likely not generated by textual analysis is because there does not appear to be a consistent grouping system among these reviews. If there was a consistent grouping system, things like the words with large positive/negative sentiments would likely perform far higher when we rank the features.

Now attempt to extract the voting score from the review (if exists), for example the first review has it.
Outline an approach.

One Approach could be, as I have already started to outline, would be to first clean the reviews as normal. This time, I will be not removing any words in order to preserve the understanding of the words. The next part would be to remove the stop words, so that we are not left with the "is" or another irrelevant word. Then we will extract the numerical value from the review pile. 

In [10]:
sentiments = dict()
sentiments[0]=[]
sentiments[1]=[]
file_path='./movie_data.csv'
with open(file_path, 'r', encoding="utf-8") as file:
    reader = csv.reader(file, delimiter=',', quotechar='"')
    next(reader)
    for line_number, line in enumerate(reader, start=1):
        if(line[1]=="0"):
            sentiments[0].append(line[0])
        if(line[1]=="1"):
            sentiments[1].append(line[0])

In [11]:
# Assuming you have a set of positive and negative words defined as pos_words and all_the_words
import pandas as pd
from nltk.corpus import stopwords
def ie_preprocess(document):
    document = re.sub('<br />', '', document)
    document = re.sub(r'[^\w\s]', ' ', document)
    sentences = [nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(document)]
    stop_words = set(stopwords.words('english'))
    return [[word for word in sent if word.lower() not in stop_words] for sent in sentences][0]

path = './movie_data.csv'
vote_df = pd.read_csv(path, encoding="utf-8")
vote_df['review'] = vote_df['review'].apply(ie_preprocess)

In [12]:
vote_df.head()

,review,sentiment
0,"[1974, teenager, Martha, Moxley, Maggie, Grace...",1
1,"[OK, really, like, Kris, Kristofferson, usual,...",0
2,"[SPOILER, read, think, watching, movie, althou...",0
3,"[hi, people, seen, wonderful, movie, im, sure,...",1
4,"[recently, bought, DVD, forgetting, much, hate...",0


In [13]:
total_set=set(['zero','one','two','three','four','five','six','seven','eight','nine','ten','eleven','10'])
def is_vote_in(list_of_reviews):
    for i,word in enumerate(list_of_reviews):
        if word.lower() == 'vote':
            if(len(list_of_reviews[i+1:i+2]) == 1):
                rating=list_of_reviews[i+1:i+2][0]
                if(rating in total_set):
                    return rating
    return False

vote_df['vote_present']=vote_df['review'].apply(is_vote_in)

In [14]:
vote_df[vote_df['vote_present'] != False]

,review,sentiment,vote_present
0,"[1974, teenager, Martha, Moxley, Maggie, Grace...",1,seven
286,"[gave, film, 2, mostly, actually, ok, cast, fi...",0,three
408,"[day, California, Presidential, Primary, midni...",1,eight
447,"[Eva, Hedy, Lamarr, got, married, older, man, ...",1,eight
456,"[suicide, father, Charlie, Kid, Davis, Ray, Bo...",0,three
...,...,...,...
49402,"[Matthew, McConaughey, mysterious, man, waitin...",1,eight
49493,"[vote, 10, 10, movie, completely, guts, Quite,...",1,10
49638,"[Julia, Kristina, Copeland, travels, husband, ...",0,four
49848,"[1967, mine, workers, find, remnants, ancient,...",0,three


In [15]:
vote_df

,review,sentiment,vote_present
0,"[1974, teenager, Martha, Moxley, Maggie, Grace...",1,seven
1,"[OK, really, like, Kris, Kristofferson, usual,...",0,False
2,"[SPOILER, read, think, watching, movie, althou...",0,False
3,"[hi, people, seen, wonderful, movie, im, sure,...",1,False
4,"[recently, bought, DVD, forgetting, much, hate...",0,False
...,...,...,...
49995,"[OK, lets, start, best, building, although, ha...",0,False
49996,"[British, heritage, film, industry, control, n...",0,False
49997,"[even, know, begin, one, family, worst, line, ...",0,False
49998,"[Richard, Tyler, little, boy, scared, everythi...",0,False
